# 0. Set up

## 0a. Directories

**Main working directory**: `/nesi/nobackup/uoa00348/boey/2023-estuarine-cazyme-diversity/`

| Item | Location | Comment |
| :--- | :--- | :--- |
| MAGs | `/nesi/project/ga02676/Waiwera_project/3.Binning/2.Final_bins` | Original set of final bins curated by Dave, Sze and Carmen. Also contains CheckM results. |
| KoFamScan<br>(version 1.3.0) | `/nesi/project/uoa02469/Software/kofam_scan_v1.3.0` | Annie updated the database on 29 Mar 2023. |
| Transporter Classification Database<br>(TCDB; 12 Apr 2023) | `/nesi/project/uoa02469/Databases/TCDB_20230412` | Boey updated database on 13 Apr 2023. |
| dbCAN<br>(version 11) | `/nesi/project/uoa02469/Databases/dbCAN2_v11` | |
| dbCAN-sub<br>(downloaded 11 Aug 2022) | `/nesi/project/uoa02469/Databases/dbCAN-sub_20220811` | |
| CAZy<br>(10 Aug 2022) | `/nesi/project/uoa02469/Databases/CAZyDB_20220806` | Compiled by Yin et al. of dbCAN. |
| SulfAtlas<br>(version 2.3.1) | `/nesi/project/uoa02469/Databases/SulfAtlas_v2.3.1` | Amino acid sequence database; HMM searchable [here](https://sulfatlas.sb-roscoff.fr/sulfatlashmm/). |
| InterProScan (version 5.61-93.0) | `bin/interproscan-5.61-93.0` | Module Java/17 needs to be loaded prior. Also, not all analyses ran, but seems to work fine for Pfam, TIGRFAM, and CDD. |


## 0b. Additional software

### InterProScan update

NeSI has InterProScan as a module, but their version is 3 years old. Best to have a newer version. 

These instructions were adapted from [here](https://interproscan-docs.readthedocs.io/en/latest/HowToDownload.html).

In [ ]:
wget --directory-prefix bin/ https://ftp.ebi.ac.uk/pub/software/unix/iprscan/5/5.61-93.0/interproscan-5.61-93.0-64-bit.tar.gz
wget --directory-prefix bin/ https://ftp.ebi.ac.uk/pub/software/unix/iprscan/5/5.61-93.0/interproscan-5.61-93.0-64-bit.tar.gz.md5

# Recommended checksum to confirm the download was successful:
md5sum -c interproscan-5.61-93.0-64-bit.tar.gz.md5
# Must return *interproscan-5.61-93.0-64-bit.tar.gz: OK*
# If not - try downloading the file again as it may be a corrupted copy.

\[2023-04-13 10:43\] `wget` is being run on a `screen` named `dl-ips`.

\[2023-04-14 14:26\] InterProScan version 5.61-93.0 installed in `bin`

### SignalP-6 GPU

Need to ask Dini to help with this. Otherwise, can continue to use CPU version, but only stick to CAZymes. Therefore, filter dbCAN identified CAZymes then run it through SignalP.

### DeepTMHMM

Potentially need to run this on their [cloud server](https://dtu.biolib.com/DeepTMHMM). This is not ideal, but I don't think I have a choice.

## 0c. Additional comments

Make sure to copy the notebook and `scripts/` to the backed up project directory at the end of the day.

I'll add a script to automate that.

Content of `scripts/0-backup_project.sh`:

```bash
#!/bin/bash -e

TARGET=/nesi/project/ga02676/Waiwera_project/boey_work/2023-estuarine-cazyme-diversity/

cp -v 2023-estuarine-cazyme-diversity.ipynb $TARGET
cp -v -r scripts/ $TARGET

```

Once there, make sure to commit to github from the project directory.

```bash
# Stage
git add --all

# Commit
git commit -m "Today's update"

# Push
git push
```

If a mistake was made:

```bash
# Reset (change HEAD back to good commit)
git reset --hard good_commit

# Push
git push -f origin <last_good_commit>:<branch>
```

## On `data/` and `results/`

`data/` is only selectively backed-up to prevent bloating. `results/` is backed-up but not pushed to the git repository.

# 1. Get high quality bins

In the MAGs directory, use CheckM stats to filter to copy MAGs with $\gt$ 70% completeness and $\lt$ 5% contamination. These bins are copied into `data/0.bins/`.

```bash
bash scripts/1.get_bins.sh
```

Contents of `1-get_bins.sh`:

In [ ]:
#!/bin/bash -e

# Subset MAGs based on CheckM stats

# Directories
DIRIN=/nesi/project/ga02676/Waiwera_project/3.Binning/2.Final_bins
DIROUT=data/1.bins

mkdir -p $DIROUT

# Variables
COMP=70 # Completeness
CONT=5  # Contamination

# Subset CheckM results and copy MAGs to data/
awk -v comp="$COMP" -v cont="$CONT" -F "\t" '($3 > comp) && ($4 < cont) {print $6}' $DIRIN/checkm_data.txt \
  | xargs -I {} cp $DIRIN/{}.fna $DIROUT/

# 2. Predict genes and annotate

Predict ORFs from MAGs using `prodigal` in an array.

```
sbatch scripts/2-ORF_prediction.sl
```

Contents of `2-ORF_prediction.sl`:

In [ ]:
#!/bin/bash -e
#SBATCH --job-name=prodigal
#SBATCH --account=uoa00348
#SBATCH --time=2:00:00
#SBATCH --mem=4G
#SBATCH --cpus-per-task=2
#SBATCH --output=slurm_out/%x.%j.%A_%a.out
#SBATCH --error=slurm_err/%x.%j.%A_%a.err
#SBATCH --array=0-250
#SBATCH --mail-type=BEGIN,END,FAIL
#SBATCH --mail-user=jian.sheng.boey@auckland.ac.nz

# Reference
printf "%s was called on %s\n" "$(basename $0)" "$(date)"

# Modules
module purge
module load prodigal/2.6.3-GCCcore-7.4.0

# Directories
DIRIN=data/1.bins
DIROUT=data/2.orf_prediction

mkdir -p $DIROUT

# Variables
ARR=($DIRIN/*.fna)
INPUT=${ARR[$SLURM_ARRAY_TASK_ID]}
NAME=$(basename $INPUT .fna)
OUTPUT=${DIROUT}/${NAME}_pred

# Run prodigal
prodigal \
  -i ${INPUT} \
  -f gff \
  -a ${OUTPUT}.faa \
  -d ${OUTPUT}.fna \
  -o ${OUTPUT}.gff \
  -p single

**Job ID:** 34268687

## 2.1 Combine and clean up predictions

Concatenate all ORF predictions then remove metadata from the ORF predictions.

```bash
bash scripts/2.1-ORF_cleanup.sh
```

Contents of `2.1-ORF_cleanup.sh`:

In [ ]:
#!/bin/bash -e

# Concatenate all ORF predictions and remove metadata from ORF predicted sequences.
# Metadata is also consolidated from the GFF files.

# Directories
DIR=data/2.orf_prediction

# Concatenate predictions and remove metadata
cat $DIR/*.faa \
  | cut -f 1 -d ' ' \
  > $DIR/allbins_pred.faa

# Create another without asterisk for interproscan
sed -e 's/\*//g' $DIR/allbins_pred.faa > $DIR/allbins_pred.noast.faa

# Consolidate GFF data
printf "bin\tnode\tsource\ttype\tstart\tend\tgff_score\tstrand\tphase\tseqid\tpartial\tstart_type\trbs_motif\trbs_spacer\tgc_cont\tconf\tscore\tcscore\tsscore\trscore\tuscore\ttscore\n" > $DIR/allbins_pred.metadata.tsv

for i in $DIR/*.gff; do
  bin=$(basename $i .gff)
  grep -v '#' $i \
    | sed -e 's/;$//' \
    | sed -e 's/;/\t/g' \
    | sed -E 's/\w+=//g' \
    | awk '{FS="\t"; OFS="\t"} {split($9, a, "_"); $1=$1"_"a[2]; print}' \
    | awk -v mag="$bin" '{FS="\t"; OFS="\t"} {print mag, $0}' \
    >> $DIR/allbins_pred.metadata.tsv
done

# 3. ORF prediction (BACKUP REQUIRED)

Predict annotations for predicted ORFs against:
* KEGG (via KoFamScan)
* dbCAN
* CAZy
* SulfAtlas
* TCDB
* InterPro

Also predict signal peptides (via SignalP-6) and transmembrane proteins (via DeepTMHMM).

Keep in mind that with InterProScan, sequences will need to be chunked to around 80,000 sequences per file.

## 3.1 InterProScan

Need to split inputs into 80,000 sequences per file.

In [ ]:
ml purge
ml SeqKit/2.2.0

mkdir -p data/tmp

seqkit split \
  data/2.orf_prediction/allbins_pred.faa \
  --out-dir data/tmp \
  --by-size 80000

Split into 10 files.

Proceed to InterPro annotations.

Contents of `3.1-interproscan.sl`:

In [ ]:
#!/bin/bash -e
#SBATCH --job-name=ipr5
#SBATCH --account=uoa00348
#SBATCH --time=2:00:00
#SBATCH --mem=24G
#SBATCH --cpus-per-task=32
#SBATCH --output=slurm_out/%x.%j.%A_%a.out
#SBATCH --error=slurm_err/%x.%j.%A_%a.err
#SBATCH --array=0-9
#SBATCH --mail-type=BEGIN,END,FAIL
#SBATCH --mail-user=jian.sheng.boey@auckland.ac.nz

# Modules
module purge
module load Java/17

# Directories
INDIR=data/tmp
OUTDIR=data/3.annotation

mkdir -p ${OUTDIR}

# Variables
interproscan=bin/interproscan-5.61-93.0/interproscan.sh

ARR=(${INDIR}/allbins_pred.part*.faa)
INFILE=${ARR[$SLURM_ARRAY_TASK_ID]}
FILEPART=$(basename ${INFILE} .faa | sed -E 's/.*(part_[0-9]*)/\1/g')
OUTBASE=${OUTDIR}/allbins_pred.interpro.${FILEPART}
APPL=Pfam,TIGRFAM,CDD
FORMAT=xml,tsv
CPU=$(($SLURM_CPUS_PER_TASK - 2))


# Run InterProScan
$interproscan \
  --applications ${APPL} \
  --cpu ${CPU} \
  --formats ${FORMAT} \
  --input ${INFILE} \
  --output-file-base ${OUTBASE}

**Job ID:** 34314370

### Back up results

XML files are compressed then copied to project directory.

TSV files are concatenated, compressed, then copied to project directory.

In [ ]:
ml purge
ml pigz

# Concatenate then compress TSV files
cat data/3.annotation/*.tsv | pigz -c -p 4 > results/allbins_pred.interpro.tsv.gz

# Archive then compress XML files
tar -cvf - data/3.annotation/*interpro.*.xml | pigz -p 4 > results/allbins_pred.interpro.xml.tar.gz

## 3.2 dbCAN

This will annotate against dbCAN and dbCAN-sub using HMMER3.

Contents of `3.2-dbcan.sl`:

In [ ]:
#!/bin/bash -e
#SBATCH --job-name=dbcan
#SBATCH --account=uoa00348
#SBATCH --time=06:00:00
#SBATCH --mem=8G
#SBATCH --cpus-per-task=32
#SBATCH --output=slurm_out/%x.%j..out
#SBATCH --error=slurm_err/%x.%j..err
#SBATCH --mail-type=BEGIN,END,FAIL
#SBATCH --mail-user=jian.sheng.boey@auckland.ac.nz

# Modules
module purge
module load HMMER/3.3.2-GCC-11.3.0

# Directories
INDIR=data/2.orf_prediction
OUTDIR=data/3.annotation
DBDIR=/nesi/project/uoa02469/Databases

# Variables 
INFILE=${INDIR}/allbins_pred.faa
INBASE=$(basename ${INFILE} .faa)

## Database for dbCAN2
DB1=${DBDIR}/dbCAN2_v11/dbCAN-HMMdb-V11
## Output for dbCAN2
OUTFILE1=${OUTDIR}/${INBASE}.dbcan.domtbl

## Database for dbCAN-sub
DB2=${DBDIR}/dbCAN-sub_20220811/dbCAN_sub.hmm
## Output for dbCAN-sub
OUTFILE2=${OUTDIR}/${INBASE}.dbcan-sub.domtbl

## dbCAN2 size 
SZDB1=$(hmmstat ${DB1} | tail -n 1 | cut -f 1 -d ' ')

## dbCAN-sub size
SZDB2=$(hmmstat ${DB2} | tail -n 1 | cut -f 1 -d ' ')

# Run
## dbCAN
hmmsearch \
  -Z $SZDB1 \
  --cpu ${SLURM_CPUS_PER_TASK} \
  --domtblout ${OUTFILE1} \
  -o /dev/null \
  $DB1 \
  $INFILE

## dbCAN-sub
hmmsearch \
  -Z $SZDB2 \
  --cpu ${SLURM_CPUS_PER_TASK} \
  --domtblout ${OUTFILE2} \
  -o /dev/null \
  $DB2 \
  $INFILE


**Job ID:** 34375960

## 3.3 KEGG (KoFamScan)

Remember to funnel the temporary files to `data/tmp/`, then have `KoFamScan` delete the temporary files.

Contents of `3.3-kofamscan.sl`:

In [ ]:
#!/bin/bash -e
#SBATCH --job-name=kofam
#SBATCH --account=uoa00348
#SBATCH --time=24:00:00
#SBATCH --mem=64G
#SBATCH --cpus-per-task=32
#SBATCH --output=slurm_out/%x.%j.out
#SBATCH --error=slurm_err/%x.%j.err
#SBATCH --mail-type=BEGIN,END,FAIL
#SBATCH --mail-user=jian.sheng.boey@auckland.ac.nz

# Modules
module purge
module load \
  Ruby/3.0.1-GCC-11.3.0 \
  HMMER/3.3.2-GCC-11.3.0 \
  Parallel/20220922

# Directories
INDIR=data/2.orf_prediction
OUTDIR=data/3.annotation
SOFTDIR=/nesi/project/uoa02469/Software/kofam_scan_v1.3.0
TMPDIR=data/tmp

# Variables
INFILE=${INDIR}/allbins_pred.faa
INBASE=$(basename ${INFILE} .faa)
OUTFILE=${OUTDIR}/${INBASE}.kofam.tsv

## Software variables
kofamscan=${SOFTDIR}/bin/exec_annotation

KOLIST=${SOFTDIR}/db/ko_list
PROFILE=${SOFTDIR}/db/profiles
FORMAT=detail-tsv

# Run
$kofamscan \
  --format=$FORMAT \
  --profile=$PROFILE \
  --ko-list=$KOLIST \
  --cpu=${SLURM_CPUS_PER_TASK} \
  --tmp-dir=$TMPDIR \
  ${INFILE} \
  > ${OUTFILE}


**Job ID:** 34372470